In [1]:
"""
Calculate fire severity statistics within AFD observations from MODIS and VIIRS.
Severity index: Composite Burn Severity (CBI), calculated in GEE via Parks (2018)

Author: maxwell.cook@colorado.edu
"""

import ee
import geemap

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

print("Success")

Success


In [5]:
# Load the FRP observations
mod61 = ee.FeatureCollection('projects/jfsp-aspen/assets/MOD61-afd_aspen-fires_2018_to_2023')
snpp = ee.FeatureCollection('projects/jfsp-aspen/assets/SNPP-afd_aspen-fires_2018_to_2023')
print(mod61.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-111.73731841616507, 39.48005872191875], [-111.72574402912765, 39.4815293785386], [-111.72767723384932, 39.490332419395216], [-111.73925298333046, 39.488861548276724], [-111.73731841616507, 39.48005872191875]]]}, 'id': '000000000000000020c7', 'properties': {'ACQ_DATE': '2018/05/17 00:00:00.000', 'ACQ_DATETI': '2018/05/17 00:00:00.000', 'ACQ_MONTH': 0, 'ACQ_TIME': '2036', 'ACQ_YEAR': 0, 'BRIGHTNESS': 319.7, 'BRIGHT_T31': 301.2, 'CONFIDENCE': 52, 'DAYNIGHT': 'D', 'FIRE_YEAR': 0, 'FRP': 9.4, 'INCIDENT': '2018', 'INSTRUMENT': 'MODIS', 'LATITUDE': 39.4852, 'LONGITUDE': -111.7325, 'OBJECTID': 9582, 'SATELLITE': 'Aqua', 'SCAN': 1, 'TRACK': 1, 'TYPE': 0, 'VERSION': '6.03', 'VID': 7298, 'fid': 1}}


In [9]:
# Load the CBI mosaic
cbi = ee.Image('projects/jfsp-aspen/assets/afd_aspen-fires_cbi_mosaic_2018_to_2023')
print(cbi.getInfo())

{'type': 'Image', 'bands': [{'id': 'dnbr', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [57374, 58043], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -120.32286443776425, 0, -0.00026949458523585647, 48.372391587739585]}, {'id': 'rbr', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [57374, 58043], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -120.32286443776425, 0, -0.00026949458523585647, 48.372391587739585]}, {'id': 'rdnbr', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [57374, 58043], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, -120.32286443776425, 0, -0.00026949458523585647, 48.372391587739585]}, {'id': 'dndvi', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [57374, 58043], 'cr